In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


##import libraries

import datetime
print("Script started at "+ str(datetime.datetime.now()))
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# In[2]:


#import last 30-days csv

data = pd.read_csv("input.csv",engine='python',skipfooter=18)

#filter for all IOs with XHK17-DC024
data = data[data['Insertion Order'].str.contains("LR")]


# In[3]:


#create custom columns

##create 'Market' column

###define extract market function

def extract_market (io):
    try:        
        return io[io.index('II_')+3:io.index('II_')+5]
    except:
        return 'NA'

def extract_mo (li):
    try:        
        return li[12:14]
    except:
        return 'NA'
    
data['Market']=data.apply(lambda row: extract_market(str(row['Insertion Order'])),axis=1)


##create 'Strategy' column

def extract_strategy (io):
    if '_RT_' in io:
        return 'Retargeting'
    else:
        return 'Prospecting'
    
def extract_sub_strategy (li):
    try:
        if li.split('_')[2]=='RTG':
            return li.split('_')[3]+' '+li.split('_')[4]
        elif li.split('_')[3]=='MGM Cotai':
            return li.split('_')[3]+' '+li.split('_')[4]
        else:
            return li.split('_')[3]
    except:
        return 'Others'

data['Strategy']=data.apply(lambda row: extract_strategy(str(row['Insertion Order'])),axis=1)

## create 'Restaurant Bookings' column

data['Restaurant Bookings']=data.apply(lambda row: row['Total Conversions'] if row['Floodlight Activity Name']=='MGM Cotai - Restaurant Booking Confirmation' else 0,axis=1)

## create 'Hotel Bookings' column

data['Hotel Bookings']=data.apply(lambda row: row['Total Conversions'] if row['Floodlight Activity Name']=='MGM Cotai - Booking Confirmation' else 0,axis=1)

## create 'Total Bookings' = Fuerza Bruta + Hotel+Restaurant bookings column

data['All Bookings']=data.apply(lambda row: row['Total Conversions'] if ((row['Floodlight Activity Name']=='HK_MGM Cotai_Tickets_Chef Nic_Choose Tickets_Seat Map')|(row['Floodlight Activity Name']=='MGM Cotai - Booking Confirmation')|(row['Floodlight Activity Name']=='MGM Cotai - Restaurant Booking Confirmation')|(row['Floodlight Activity Name']=='HK_MGM Cotai_Tickets_Fuerza Bruta_Buy Tickets_Choose Tickets')|(row['Floodlight Activity Name']=='MGM Cotai_Tickets_Fuerza Bruta_Buy Tickets_Choose Tickets_Seat Map')) else 0,axis=1)

##create 'Chef Nic Bookings' column
data['Chef Nic Bookings']=data.apply(lambda row: row['Total Conversions'] if row['Floodlight Activity Name']=='HK_MGM Cotai_Tickets_Chef Nic_Choose Tickets_Seat Map' else 0,axis=1)

##create 'Check-outs' column

data['Check-outs']=data.apply(lambda row: row['Total Conversions'] if row['Floodlight Activity Name']=='MGM Cotai - Check Out' else 0,axis=1)

##create 'Fuerza Bruta Bookings' column
data['Fuerza Bruta']=data.apply(lambda row: row['Total Conversions'] if ((row['Floodlight Activity Name']=='HK_MGM Cotai_Tickets_Fuerza Bruta_Buy Tickets_Choose Tickets')|(row['Floodlight Activity Name']=='MGM Cotai_Tickets_Fuerza Bruta_Buy Tickets_Choose Tickets_Seat Map')) else 0,axis=1)

##create 'Week' column

def get_week (d):
    year=int(d.split('/')[0])
    month=int(d.split('/')[1])
    day=int(d.split('/')[2])
    dat=date(year, month,day)
    return dat.isocalendar()[1]
    
data['Week']=data.apply(lambda row:get_week(row['Date']),axis=1)

##create 'Sub-Strategy' column

data['Sub-strategy']=data.apply(lambda row: extract_sub_strategy(row['Line Item']),axis=1)


##create market pivot table

market_pivot=data.pivot_table(index='Market',values=['All Bookings','Check-outs','Revenue (Adv Currency)'],aggfunc=np.sum)
market_pivot['CPA']=market_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
market_pivot['CPI']=market_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
pd.options.display.float_format = '{:,.2f}'.format
#market_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
#market_pivot.fillna(value=0, inplace=True)


# In[4]:


##overall budget adjustment based on last-week data

##create last week's market pivot table


def get_lastweek():
    one_week=timedelta(weeks=1)
    return (date.today()-one_week).isocalendar()[1]
#def get_lastweek():
#    one_week=timedelta(weeks=1)
#    return date.today().isocalendar()[1]

market_pivot=data[data['Week']==get_lastweek()].pivot_table(index='Market',values=['All Bookings','Hotel Bookings','Restaurant Bookings','Fuerza Bruta','Chef Nic Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
market_pivot['CPA']=market_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
market_pivot['CPI']=market_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
market_pivot['CPTC']=market_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
pd.options.display.float_format = '{:,.2f}'.format
market_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
market_pivot.fillna(value=0, inplace=True)

CPA_table = market_pivot['CPA'][market_pivot['CPA']!=0]
CPI_table = market_pivot['CPI'][market_pivot['CPI']!=0]
CPTC_table = market_pivot['CPTC'][market_pivot['CPTC']!=0]
bookings_table = market_pivot['All Bookings']
checkouts_table = market_pivot['Check-outs']
revenue_table = market_pivot['Revenue (Adv Currency)']
markets_table = market_pivot.index.get_values()

def get_quantile(table,tier):
    return table.quantile(tier)

def get_upweighted_markets(table,metric,event):
    value_list = table[metric][table[metric]!=0]
    total_spend = table['Revenue (Adv Currency)'].sum()
    global_spend_threshold=total_spend*0.20
    intermediate_table=table[table['Revenue (Adv Currency)']>global_spend_threshold]
    intermediate_table2=intermediate_table[intermediate_table[metric]<=get_quantile(intermediate_table[metric],0.2)]
    upweighted_markets_arr = intermediate_table2.index[(intermediate_table2[event]!=0)].get_values()
    if len(table)==1:
        upweighted_markets_arr=np.array([])
        return upweighted_markets_arr
    else:
        return upweighted_markets_arr
    

def get_average_weekly_spend(value, dimension):
    intermediate_table = data[data[dimension]==value]
    return intermediate_table.groupby(['Week'])['Revenue (Adv Currency)'].sum().mean()

def get_downweighted_markets(table,metric,event,dimension):
    value_list = table[metric][table[metric]!=0]
    total_spend = table['Revenue (Adv Currency)'].sum()
    global_spend_threshold=total_spend*0.20
    intermediate_table=table[table['Revenue (Adv Currency)']>global_spend_threshold]
    intermediate_table2=intermediate_table[intermediate_table[metric]>get_quantile(intermediate_table[metric],0.8)]
    downweighted_markets_arr = intermediate_table2.index[(intermediate_table2[event]!=0)].get_values()
    zero_event_markets = table[table[event]==0].copy()
    zero_event_markets['spend_threshold']=zero_event_markets.apply(lambda row: get_average_weekly_spend(row.name,dimension),axis=1)
    if (dimension=='Market')|(dimension=='Strategy'):
        additional_downweighted_markets = zero_event_markets.index[zero_event_markets['Revenue (Adv Currency)']>zero_event_markets['spend_threshold']].get_values()
    else:
        additional_downweighted_markets = zero_event_markets.index[zero_event_markets['Revenue (Adv Currency)']>global_spend_threshold].get_values()    
    result=np.concatenate((downweighted_markets_arr,additional_downweighted_markets),axis=None)
    return result




nBookings = market_pivot['All Bookings'].sum()
nCheckouts = market_pivot['Check-outs'].sum()
nTC = market_pivot['Total Conversions'].sum()

##if there are 0 bookings look at Check-outs to decide which markets to upweight

if nBookings==0:
    if nCheckouts!=0: 
        print('bookings = 0 checkouts > 0')
        upweighted_markets_arr=get_upweighted_markets(market_pivot,'CPI','Check-outs')
        downweighted_markets_arr=get_downweighted_markets(market_pivot,'CPI','Check-outs','Market')
    else:
        print('bookings = 0 checkouts = 0')
        upweighted_markets_arr=get_upweighted_markets(market_pivot,'CPTC','Total Conversions')
        downweighted_markets_arr=get_downweighted_markets(market_pivot,'CPTC','Total Conversions','Market')
else:
    print('bookings > 0')
    upweighted_markets_arr=get_upweighted_markets(market_pivot,'CPA','All Bookings')
    downweighted_markets_arr=get_downweighted_markets(market_pivot,'CPA','All Bookings','Market')

    
name=str(datetime.datetime.today())
name=name[0:10]+".txt"    
file = open(name,"w")
    
for row in downweighted_markets_arr:
    file.write('Reduced overall delivery in ' +row)
    print('Reduced overall delivery in ' +row)


for row in upweighted_markets_arr:
    file.write('Increased overall delivery in ' +row)
    print('Increased overall delivery in ' +row)
    
    


# In[5]:


##strategy level optimisation notes

for row in market_pivot.index:
    device_pivot=data[(data['Week']==get_lastweek())&(data['Market']==row)].pivot_table(index='Strategy',values=['All Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
    device_pivot['CPA']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
    device_pivot['CPI']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
    device_pivot['CPTC']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
    pd.options.display.float_format = '{:,.2f}'.format
    nBookings = device_pivot['All Bookings'].sum()
    nCheckouts = device_pivot['Check-outs'].sum()
    device_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
    device_pivot.fillna(value=0, inplace=True)
    
    if nBookings!=0:
        overall_cpx = market_pivot.loc[row,'CPA']
    elif nCheckouts!=0:
        overall_cpx = market_pivot.loc[row,'CPI']
    else:
        overall_cpx = market_pivot.loc[row,'CPTC']
    
    if nBookings==0:
        if nCheckouts!=0: 
            loop='Check-outs'            
            upweightedStrat=get_upweighted_markets(device_pivot,'CPI','Check-outs')
            downweightedStrat=get_downweighted_markets(device_pivot,'CPI','Check-outs','Strategy')
        else:
            loop='All events'
            upweightedStrat=get_upweighted_markets(device_pivot,'CPTC','Total Conversions')
            downweightedStrat=get_downweighted_markets(device_pivot,'CPTC','Total Conversions','Strategy')
    else:
        loop='Bookings'
        upweightedStrat=get_upweighted_markets(device_pivot,'CPA','All Bookings')
        downweightedStrat=get_downweighted_markets(device_pivot,'CPA','All Bookings','Strategy')

    device_pivot=data[(data['Week']==get_lastweek())&(data['Market']==row)].pivot_table(index='Region',values=['All Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
    device_pivot['CPA']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
    device_pivot['CPI']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
    device_pivot['CPTC']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
    pd.options.display.float_format = '{:,.2f}'.format
    nBookings = device_pivot['All Bookings'].sum()
    nCheckouts = device_pivot['Check-outs'].sum()
    device_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
    device_pivot.fillna(value=0, inplace=True)
    
    if nBookings==0:
        if nCheckouts!=0: 
            loop='Check-outs'
            upweighted_region=get_upweighted_markets(device_pivot,'CPI','Check-outs')
            downweighted_region=get_downweighted_markets(device_pivot,'CPI','Check-outs','Region')
        else:
            loop='All events'
            upweighted_region=get_upweighted_markets(device_pivot,'CPTC','Total Conversions')
            downweighted_region=get_downweighted_markets(device_pivot,'CPTC','Total Conversions','Region')
    else:
        loop='Bookings'
        upweighted_region=get_upweighted_markets(device_pivot,'CPA','All Bookings')
        downweighted_region=get_downweighted_markets(device_pivot,'CPA','All Bookings','Region')
        
        
    device_pivot=data[(data['Week']==get_lastweek())&(data['Market']==row)].pivot_table(index='Device Type',values=['All Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
    device_pivot['CPA']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
    device_pivot['CPI']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
    device_pivot['CPTC']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
    pd.options.display.float_format = '{:,.2f}'.format
    nBookings = device_pivot['All Bookings'].sum()
    nCheckouts = device_pivot['Check-outs'].sum()
    device_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
    device_pivot.fillna(value=0, inplace=True)
    
    if nBookings==0:
        if nCheckouts!=0: 
            loop='Check-outs'
            upweightedDevice=get_upweighted_markets(device_pivot,'CPI','Check-outs')
            downweightedDevice=get_downweighted_markets(device_pivot,'CPI','Check-outs','Device Type')
        else:
            loop='All events'
            upweightedDevice=get_upweighted_markets(device_pivot,'CPTC','Total Conversions')
            downweightedDevice=get_downweighted_markets(device_pivot,'CPTC','Total Conversions','Device Type')
    else:
        loop='Bookings'
        upweightedDevice=get_upweighted_markets(device_pivot,'CPA','All Bookings')
        downweightedDevice=get_downweighted_markets(device_pivot,'CPA','All Bookings','Device Type')
    
    device_pivot=data[(data['Week']==get_lastweek())&(data['Market']==row)].pivot_table(index='Browser',values=['All Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
    device_pivot['CPA']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
    device_pivot['CPI']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
    device_pivot['CPTC']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
    pd.options.display.float_format = '{:,.2f}'.format
    nBookings = device_pivot['All Bookings'].sum()
    nCheckouts = device_pivot['Check-outs'].sum()
    device_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
    device_pivot.fillna(value=0, inplace=True)
    
    if nBookings==0:
        if nCheckouts!=0: 
            loop='Check-outs'
            upweightedBrowser=get_upweighted_markets(device_pivot,'CPI','Check-outs')
            downweightedBrowser=get_downweighted_markets(device_pivot,'CPI','Check-outs','Browser')
        else:
            loop='All events'
            upweightedBrowser=get_upweighted_markets(device_pivot,'CPTC','Total Conversions')
            downweightedBrowser=get_downweighted_markets(device_pivot,'CPTC','Total Conversions','Browser')
    else:
        loop='Bookings'
        upweightedBrowser=get_upweighted_markets(device_pivot,'CPA','All Bookings')
        downweightedBrowser=get_downweighted_markets(device_pivot,'CPA','All Bookings','Browser')

    device_pivot=data[(data['Week']==get_lastweek())&(data['Market']==row)].pivot_table(index='Sub-strategy',values=['All Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
    device_pivot['CPA']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
    device_pivot['CPI']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Check-outs'] if row['Check-outs']!=0 else 0,axis=1)
    device_pivot['CPTC']=device_pivot.apply(lambda row: row['Revenue (Adv Currency)'] / row['Total Conversions'] if row['Total Conversions']!=0 else 0,axis=1)
    pd.options.display.float_format = '{:,.2f}'.format
    nBookings = device_pivot['All Bookings'].sum()
    nCheckouts = device_pivot['Check-outs'].sum()
    device_pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
    device_pivot.fillna(value=0, inplace=True)
    
    if nBookings==0:
        if nCheckouts!=0: 
            loop='Check-outs'
            upweighted_substrategy=get_upweighted_markets(device_pivot,'CPI','Check-outs')
            downweighted_substrategy=get_downweighted_markets(device_pivot,'CPI','Check-outs','Sub-strategy')
        else:
            loop='All events'
            upweighted_substrategy=get_upweighted_markets(device_pivot,'CPTC','Total Conversions')
            downweighted_substrategy=get_downweighted_markets(device_pivot,'CPTC','Total Conversions','Sub-strategy')
    else:
        loop='Bookings'
        upweighted_substrategy=get_upweighted_markets(device_pivot,'CPA','All Bookings')
        downweighted_substrategy=get_downweighted_markets(device_pivot,'CPA','All Bookings','Sub-strategy')
    
        
        
    print('XXXXXXXXXXXXXX')
    file.write("XXXXXXXXXXXXXX\n\n")
    print()
    print('Market: '+row)
    file.write('Market: '+row+"\n")
    print("Overall CPX: " + format(overall_cpx,'.2f')+"\n")
    file.write("Overall CPX: " + format(overall_cpx,'.2f')+"\n")
    print("X (Success Metric): "+loop+"\n")
    file.write("X (Success Metric): "+loop+"\n")
    print()
    file.write("\n")
    i = 0
    if upweightedDevice.size>0:
        print("Up-weighted Device Type(s):")
        file.write("Up-weighted Device Type(s):\n")
        while i < upweightedDevice.size:
            print("  o "+upweightedDevice[i])
            file.write("  o "+upweightedDevice[i]+"\n")
            i+=1
        print()
        file.write("\n")
        
    i = 0
    if downweightedDevice.size>0:
        print("Down-weighted Device Type(s):")
        file.write("Down-weighted Device Type(s):\n")
        while i < downweightedDevice.size:
            print("  o "+downweightedDevice[i])
            file.write("  o "+downweightedDevice[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i = 0
    if upweightedBrowser.size>0:
        print("Up-weighted Browser(s):")
        file.write("Up-weighted Browser(s):\n")
        while i < upweightedBrowser.size:
            print("  o "+upweightedBrowser[i])
            file.write("  o "+upweightedBrowser[i]+"\n")
            i+=1
        print()  
        file.write("\n")
        
    i = 0
    if downweightedBrowser.size>0:
        print("Down-weighted Browser(s):")
        file.write("Down-weighted Browser(s):\n")
        while i < downweightedBrowser.size:
            print("  o "+downweightedBrowser[i])
            file.write("  o "+downweightedBrowser[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if upweightedStrat.size>0:
        print("Up-weighted Strategy:")
        file.write("Up-weighted Strategy:\n")
        while i < upweightedStrat.size:
            print("  o "+upweightedStrat[i])
            file.write("  o "+upweightedStrat[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if downweightedStrat.size>0:
        print("Down-weighted Strategy:")
        file.write("Down-weighted Strategy:\n")
        while i < downweightedStrat.size:
            print("  o "+downweightedStrat[i])
            file.write("  o "+downweightedStrat[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if upweighted_region.size>0:
        print("Up-weighted Region(s):")
        file.write("Up-weighted Region(s):\n")
        while i < upweighted_region.size:
            print("  o "+upweighted_region[i])
            file.write("  o "+upweighted_region[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if downweighted_region.size>0:
        print("Down-weighted Region(s):")
        file.write("Down-weighted Region(s):\n")
        while i < downweighted_region.size:
            print("  o "+downweighted_region[i])
            file.write("  o "+downweighted_region[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if upweighted_substrategy.size>0:
        print("Up-weighted Segment(s):")
        file.write("Up-weighted Segment(s):\n")
        while i < upweighted_substrategy.size:
            print("  o "+upweighted_substrategy[i])
            file.write("  o "+upweighted_substrategy[i]+"\n")
            i+=1
        print()
        file.write("\n")
    i=0
    if downweighted_substrategy.size>0:
        print("Down-weighted Segment(s):")
        file.write("Down-weighted Segment(s):\n")
        while i < downweighted_substrategy.size:
            print("  o "+downweighted_substrategy[i])
            file.write("  o "+downweighted_substrategy[i]+"\n")
            i+=1
        print()
        file.write("\n\n")
        
        
file.close()


# In[6]:


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.catplot(x='Market', y="Revenue (Adv Currency)", kind="bar", data=market_pivot.reset_index())
ax2 = plt.twinx()
sns.lineplot(data=market_pivot[['CPA']], color="black",ax=ax2)
ax2.set_ylim(bottom=0)

plt.savefig('last_week_overview.png',bbox_inches = "tight")


# In[7]:


last_week=data[(data['Week']==get_lastweek())].pivot_table(index=['Market','Strategy'],values=['All Bookings','Hotel Bookings','Restaurant Bookings','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
last_week['CPA']=last_week.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
last_week


# In[8]:


week_table=data[data['Week']>get_lastweek()-4].pivot_table(index=['Market','Week'],values=['All Bookings','Hotel Bookings','Restaurant Bookings','Check-outs','Total Conversions','Revenue (Adv Currency)'],aggfunc=np.sum)
week_table['CPA']=week_table.apply(lambda row: row['Revenue (Adv Currency)'] / row['All Bookings'] if row['All Bookings']!=0 else 0,axis=1)
week_table


# In[10]:


market_pivot.to_csv('market_summary.csv')


# In[9]:


print("Script completed at "+ str(datetime.datetime.now()))


# In[ ]:




